In [1]:
from IPython.display import display_html
from itertools import product
from math import sin, cos
import pandas as pd
import numpy as np
from z3 import *

In [2]:
V_STOP, V_LOW, V_HIGH = 0, 1, 10 # velocidades dos modos baixo e alto, em m
L = 1e3 # Lado dos setores em m
N = 10 # Numero de setores
THETA = 15 # angulo de viragem entre modos
TAU = 3 # tempo minimo entre transicoes timed
ALL_ROUTES = [i*THETA for i in range(int(360/THETA))] # todos os ângulos possíveis
ALL_VELS = [V_STOP, V_LOW, V_HIGH] # todas as velocidades possiveis

Mode, (M_STOP, M_LOW, M_HIGH) = EnumSort("Mode", ("V_STOP", "V_LOW", "V_HIGH"))

In [3]:
deg_to_rad = lambda a: a * np.pi / 180
vel_to_mode = lambda v: M_LOW if (v == V_LOW) else M_HIGH if (v == V_HIGH) else M_STOP
mode_to_vel = lambda m: V_LOW if (m == M_LOW) else V_HIGH if (m == M_HIGH) else V_STOP
z3tofloat = lambda v: float(v.numerator_as_long())/float(v.denominator_as_long())

Considere-se um lago com $N \times N$ setores, cada um de lado $L$. Podemos descrever um conjunto de barcos $\mathcal{B}$ e o controlador através de um FOTS. Cada barco pode tomar um conjunto de rotas $\mathcal{R} = \{ 15, 30, \dots, 345 \}$ e um conjunto de velocidades $\mathcal{V} = \{\mathtt{stop}, \mathtt{low}, \mathtt{high}\}$

Sejam $\tilde{b} \in \mathcal{B}$ um barco aleatório e $\tilde{\varphi} \in \mathcal{R}$, $0 \le \tilde{\mathtt{sx}} < N$ e $0 \le \tilde{\mathtt{sy}} < N$ inteiros com valores aleatórios. Podemos então definir a inicialização do traço:

$$ \mathtt{init}(\alpha) \; \equiv \; \forall_{b \in \mathcal{B}} \; \big( v_b = \mathtt{high} \; \wedge \; \varphi_b = \tilde{\varphi} \; \wedge \; \mathtt{sx}_b = \tilde{\mathtt{sx}} \; \wedge \; \mathtt{sy}_b = \tilde{\mathtt{sy}} \; \wedge \; L \cdot \mathtt{sx}_b \le x_b \le L \cdot (\mathtt{sx}_b+1) \; \wedge \; L \cdot \mathtt{sy}_b \le y_b \le L \cdot (\mathtt{sy}_b+1) \; \wedge \; t_b = 0 \big) \; \wedge \; v_c = v_{\tilde{b}} \; \wedge \; \varphi_c = \varphi_{\tilde{b}} \; \wedge \; \mathtt{sx}_c = \mathtt{sx}_{\tilde{b}} \; \wedge \; \mathtt{sy}_c = \mathtt{sy}_{\tilde{b}}$$

In [4]:
def declare(i, num_boats=3):
    trace = {}
    for j in range(num_boats):
        trace[f"b{j}"] = {}
        trace[f"b{j}"]["v"] = Const(f"b{j}_v{i}", Mode)
        trace[f"b{j}"]["a"] = Int(f"b{j}_a{i}")
        trace[f"b{j}"]["sx"] = Int(f"b{j}_sx{i}")
        trace[f"b{j}"]["sy"] = Int(f"b{j}_sy{i}")
        trace[f"b{j}"]["x"] = Real(f"b{j}_x{i}")
        trace[f"b{j}"]["y"] = Real(f"b{j}_y{i}")
        trace[f"b{j}"]["t"] = Real(f"b{j}_t{i}")
        
    trace["c"] = {}
    trace["c"]["v"] = Const(f"c_v{i}", Mode)
    trace["c"]["a"] = Int(f"c_a{i}")
    trace["c"]["sx"] = Int(f"c_sx{i}")
    trace["c"]["sy"] = Int(f"c_sy{i}")
    
    return trace
        
        
def init(trace):
    r = []
    for b in trace:
        if "b" in b:
            # Inicializar cada um dos barcos
            r.append(trace[b]["v"] == M_HIGH)
            index = np.random.randint(len(ALL_ROUTES))
            r.append(trace[b]["a"] == ALL_ROUTES[index])
            # r.append(trace[b]["sx"] == np.random.randint(N))
            # r.append(trace[b]["sy"] == np.random.randint(N))
            r.append(And(trace[b]["x"] >= L*trace[b]["sx"], trace[b]["x"] <= L*(trace[b]["sx"]+1)))
            r.append(And(trace[b]["y"] >= L*trace[b]["sy"], trace[b]["y"] <= L*(trace[b]["sy"]+1)))
            r.append(trace[b]["t"] == 0)
            
    r.append(trace["b0"]["sx"] == 1)
    r.append(trace["b0"]["sy"] == 1)
    r.append(trace["b1"]["sx"] == 3)
    r.append(trace["b1"]["sy"] == 1)
    r.append(trace["b2"]["sx"] == 4)
    r.append(trace["b2"]["sy"] == 2)
            
    # Inicializar o controlador com o estado um barco aleatório
    index = np.random.randint(len(trace)-1)
    r.append(trace["c"]["v"] == trace[f"b{index}"]["v"])
    r.append(trace["c"]["a"] == trace[f"b{index}"]["a"])
    r.append(trace["c"]["sx"] == trace[f"b{index}"]["sx"])
    r.append(trace["c"]["sy"] == trace[f"b{index}"]["sy"])
            
    return And(r)

## Transições das variáveis contínuas

Dado dois traços $\alpha$ e $\alpha'$ e um barco $b \in \mathcal{B}$ podemos definir a atualização da sua posição:

$$ \mathtt{updt\_pos}(\alpha, \alpha', b) \; \equiv \; \forall_{v \in \mathcal{V}, \varphi \in \mathcal{R}}, \; v_b = v \; \wedge \; \varphi_b = \varphi \; \wedge \; x' = x + v \cdot \cos \varphi \cdot (t'-t) \; \wedge \; y' = y + v \cdot \sin \varphi \cdot (t'-t) $$

Tendo em conta que há três velocidades distintas para cada barco, temos de ter em conta todas as transições $\mathtt{high} \leftrightarrow \mathtt{low}$ e $\mathtt{low} \leftrightarrow \mathtt{stop}$, bem como as transições onde a velocidade fica inalterada:

$$ \mathtt{high\_high}(\alpha, \alpha', b) \; \equiv \; v_b = \mathtt{high} \; \wedge \; v_b' = \mathtt{high} \; \wedge \; t' > t \; \wedge \; \mathtt{update\_pos}(\alpha, \alpha', b) $$

$$ \mathtt{high\_low}(\alpha, \alpha', b) \; \equiv \; v_b = \mathtt{high} \; \wedge \; v_b' = \mathtt{low} \; \wedge \; t' = t + 500 \; \wedge \; \mathtt{update\_pos}(\alpha, \alpha', b) $$

$$ \mathtt{low\_high}(\alpha, \alpha', b) \; \equiv \; v_b = \mathtt{low} \; \wedge \; v_b' = \mathtt{high} \; \wedge \; t' = t + 500 \; \wedge \;\mathtt{update\_pos}(\alpha, \alpha', b) $$

$$ \mathtt{low\_low}(\alpha, \alpha', b) \; \equiv \; v_b = \mathtt{low} \; \wedge \; v_b' = \mathtt{low} \; \wedge \; t' > t \; \wedge \; \mathtt{update\_pos}(\alpha, \alpha', b) $$

$$ \mathtt{low\_stop}(\alpha, \alpha', b) \; \equiv \; v_b = \mathtt{low} \; \wedge \; v_b' = \mathtt{stop} \; \wedge \; t' = t + 50 \; \wedge \; \mathtt{update\_pos}(\alpha, \alpha', b) $$

$$ \mathtt{stop\_low}(\alpha, \alpha', b) \; \equiv \; v_b = \mathtt{stop} \; \wedge \; v_b' = \mathtt{low} \; \wedge \; t' = t + 50 \; \wedge \; \mathtt{update\_pos}(\alpha, \alpha', b) $$

$$ \mathtt{stop\_stop}(\alpha, \alpha', b) \; \equiv \; v_b = \mathtt{stop} \; \wedge \; v_b' = \mathtt{stop} \; \wedge \; t' > t \; \wedge \; \mathtt{update\_pos}(\alpha, \alpha', b) $$

Assim sendo, todas as mudanças de velocidade bem como posição e tempo de cada barco pode ser definida pela disjunção destas transições:

$$ \mathtt{vel\_trans}(\alpha, \alpha', b) = \mathtt{high\_high}(\alpha, \alpha', b) \; \vee \; \mathtt{high\_low}(\alpha, \alpha', b) \; \vee \; \mathtt{low\_high}(\alpha, \alpha', b) \; \vee \; \mathtt{low\_low}(\alpha, \alpha', b) \; \vee \; \mathtt{low\_stop}(\alpha, \alpha', b) \; \vee \; \mathtt{stop\_low}(\alpha, \alpha', b) \; \vee \; \mathtt{stop\_stop}(\alpha, \alpha', b) $$

In [5]:
def update_pos(prev, curr, b):
    r = []
    modes = list(product(*[ALL_ROUTES, ALL_VELS]))
    
    for a, v in modes:
        vel = prev[b]["v"] == vel_to_mode(v)
        ang = prev[b]["a"] == a
        x = curr[b]["x"] == prev[b]["x"] + v * cos(deg_to_rad(a)) * (curr[b]["t"]-prev[b]["t"])
        y = curr[b]["y"] == prev[b]["y"] + v * sin(deg_to_rad(a)) * (curr[b]["t"]-prev[b]["t"])
        r.append(And(x, y, ang, vel))
    
    return Or(r)

def high_high(prev, curr, b):
    r = []
    
    r.append(And(prev[b]["v"] == M_HIGH, curr[b]["v"] == M_HIGH))
    r.append(curr[b]["t"] > prev[b]["t"])
    r.append(update_pos(prev, curr, b))
    
    return And(r)

def high_low(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_HIGH, curr[b]["v"] == M_LOW))
    r.append(curr[b]["t"] == prev[b]["t"] + 500)
    r.append(update_pos(prev, curr, b))
    
    return And(r)
    
def low_high(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_LOW, curr[b]["v"] == M_HIGH))
    r.append(curr[b]["t"] == prev[b]["t"] + 500)
    r.append(update_pos(prev, curr, b))
    
    return And(r)

def low_low(prev, curr, b):
    r = []
    
    r.append(And(prev[b]["v"] == M_LOW, curr[b]["v"] == M_LOW))
    r.append(curr[b]["t"] > prev[b]["t"])
    r.append(update_pos(prev, curr, b))
    
    return And(r)
    
def low_stop(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_LOW, curr[b]["v"] == M_STOP))
    r.append(curr[b]["t"] == prev[b]["t"] + 50)
    r.append(update_pos(prev, curr, b))
    
    return And(r)
    
def stop_low(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_STOP, curr[b]["v"] == M_LOW))
    r.append(curr[b]["t"] == prev[b]["t"] + 50)
    r.append(update_pos(prev, curr, b))
    
    return And(r)

def stop_stop(prev, curr, b):
    r = []
    
    r.append(And(prev[b]["v"] == M_STOP, curr[b]["v"] == M_STOP))
    r.append(curr[b]["t"] > prev[b]["t"])
    r.append(update_pos(prev, curr, b))
    
    return And(r)

## Untimed

Considerações:

- Quando um barco muda de setor e não está em perigo, ele continua no mesmo modo

Para uma transição untimed ocorrer para algum dos barcos, então este deve mudar de setor. Podemos verificar se um barco muda de setor verificando se este transitou para setores adjacentes:

$$ \mathtt{sector\_change}(\alpha, \alpha', b) \; \equiv \; \big( \mathtt{sx}_b' = \mathtt{sx}_b + 1 \; \wedge \; \mathtt{sy}_b' = \mathtt{sy}_b \big) \; \vee \; \big( \mathtt{sx}_b' = \mathtt{sx}_b - 1 \; \wedge \; \mathtt{sy}_b' = \mathtt{sy}_b \big) \; \vee \; \big( \mathtt{sx}_b' = \mathtt{sx}_b \; \wedge \; \mathtt{sy}_b' = \mathtt{sy}_b + 1 \big) \; \vee \; \big( \mathtt{sx}_b' = \mathtt{sx}_b \; \wedge \; \mathtt{sy}_b' = \mathtt{sy}_b - 1 \big) $$

Quando um barco faz uma transição untimed, este fica com o modo e velocidade do controlador. Podemos agora definir uma transição untimed para os barcos:
    
$$ \mathtt{untimedb}(\alpha, \alpha', b) \; \equiv \; \mathtt{vel\_trans}(\alpha, \alpha', b) \; \wedge \; 0 \le \mathtt{sx}_b' < N \; \wedge \; 0 \le \mathtt{sy}' < N \; \wedge \; \mathtt{sx}_b' = x_b'/L \; \wedge \; \mathtt{sy}_b' = y_b'/L \; \wedge \; \mathtt{sector\_change}(\alpha, \alpha', b) \; \wedge \; \mathtt{v}_b' = \mathtt{v}_c' \; \wedge \; \mathtt{\varphi}_b' = \mathtt{\varphi}_c' $$

In [6]:
def sector_change(prev, curr, b):
    left = And(curr[b]["sx"]==prev[b]["sx"]-1, curr[b]["sy"]==prev[b]["sy"])
    right = And(curr[b]["sx"]==prev[b]["sx"]+1, curr[b]["sy"]==prev[b]["sy"])
    up = And(curr[b]["sx"]==prev[b]["sx"], curr[b]["sy"]==prev[b]["sy"]+1)
    down = And(curr[b]["sx"]==prev[b]["sx"], curr[b]["sy"]==prev[b]["sy"]-1)
    return Or(left, right, up, down)

def b_untimed(prev, curr, b):
    # Mudança das variáveis contínuas do barco
    low2stop, low2low, low2high = low_stop(prev, curr, b), low_low(prev, curr, b), low_high(prev, curr, b)
    high2low, high2high = high_low(prev, curr, b), high_high(prev, curr, b)
    xyt_evolve = Or(low2stop, low2low, low2high, high2low, high2high)
    
    # Mudança do setor do barco
    sx = And(curr[b]["sx"] == curr[b]["x"] / L, curr[b]["sx"]>=0, curr[b]["sx"]<N)
    sy = And(curr[b]["sy"] == curr[b]["y"] / L, curr[b]["sy"]>=0, curr[b]["sy"]<N)
    sector = And(sx, sy)
    
    # Mundaça de modo do barco de acordo com o controlador
    change = sector_change(prev, curr, b)
    vel_ang = And(curr[b]["v"] == curr["c"]["v"], curr[b]["a"] == curr["c"]["a"])
    
    return And(xyt_evolve, sector, change, vel_ang)

Para realizar a transição untimed do controlador precisamos de saber avaliar duas coisas:
- Quando um barco está em perigo
- Quando dois barcos podem colidir - estão em posições adjacentes

Podemos avaliar se dois barcos estão em perigo da seguinte forma:

$$ \mathtt{danger\_boats}(\alpha, b_1, b_2) \; \equiv \; \mathtt{sx}_{b_2} - 2 \le \mathtt{sx}_{b_1} \le \mathtt{sx}_{b_2} + 2 \; \wedge \; \mathtt{sy}_{b_2} - 2 \le \mathtt{sy}_{b_1} \le \mathtt{sy}_{b_2} + 2 $$

Logo, para um barco se encontrar em perigo:

$$ \mathtt{danger} (\alpha, b) \; \equiv \; \bigvee_{b' \in \mathcal{B} \setminus b} \mathtt{danger\_boats} (\alpha, b, b') $$

É também necessário definir uma condição lógica que verifique se um barco tem outros barcos adjacentes, uma vez que se este for o caso, a sua velocidade deve ser modificada pelo controlador:

$$ \mathtt{adjacent\_boats} (\alpha, b_1, b_2) \; \equiv \; \left( \mathtt{sx}_{b_1} = \mathtt{sx}_{b_2} - 1 \; \wedge \; \mathtt{sy}_{b_1} = \mathtt{sy}_{b_2} \right) \; \vee \; \left( \mathtt{sx}_{b_1} = \mathtt{sx}_{b_2} + 1 \; \wedge \; \mathtt{sy}_{b_1} = \mathtt{sy}_{b_2} \right) \; \vee \; \left( \mathtt{sx}_{b_1} = \mathtt{sx}_{b_2} \; \wedge \; \mathtt{sy}_{b_1} = \mathtt{sy}_{b_2} - 1 \right) \; \vee \; \left( \mathtt{sx}_{b_1} = \mathtt{sx}_{b_2} \; \wedge \; \mathtt{sy}_{b_1} = \mathtt{sy}_{b_2} + 1 \right) $$

$$ \mathtt{adjacent} (\alpha, b) \; \equiv \; \bigvee_{b' \in \mathcal{B} \setminus b} \mathtt{adjacent\_boats} (\alpha, b, b') $$

In [7]:
def danger_boats(trace, b1, b2):
    sx = And(trace[b1]["sx"]<=trace[b2]["sx"]+2, trace[b1]["sx"]>=trace[b2]["sx"]-2)
    sy = And(trace[b1]["sy"]<=trace[b2]["sy"]+2, trace[b1]["sy"]>=trace[b2]["sy"]-2)
    return And(sx, sy)

danger = lambda trace, b: Or([danger_boats(trace, b_, b) for b_ in trace if ("b" in b_ and b!=b_)])

def adjacent_boats(trace, b1, b2):
    left  = And(trace[b1]["sx"]==trace[b2]["sx"]-1, trace[b1]["sy"]==trace[b2]["sy"])
    right = And(trace[b1]["sx"]==trace[b2]["sx"]+1, trace[b1]["sy"]==trace[b2]["sy"])
    up    = And(trace[b1]["sx"]==trace[b2]["sx"], trace[b1]["sy"]==trace[b2]["sy"]+1)
    down  = And(trace[b1]["sx"]==trace[b2]["sx"], trace[b1]["sy"]==trace[b2]["sy"]-1)
    
    return Or (left, right, up, down)

adjacent = lambda trace, b: Or([adjacent_boats(trace, b, b_) for b_ in trace if ("b" in b_ and b!=b_)])

A verificação de perigo e de adjacência é suficiente para definir a velocidade do controlador, de modo a ajustar a velocidade do barco que realizar uma transição, mas não é suficiente para inferir que rota este deve tomar. Para determinar esta rota, é necessário ainda verificar se o barco que muda de setor *pode* transitar para um setor onde outro barco possa transitar, isto é, se algum barco se encontra numa posição adjacente às posições adjacentes do barco que muda de setor. Caso tal seja verdade, a sua rota deve ser mudada para este viajar para um setor seguro.

Para verificar isto, definam-se então:

$$ \mathtt{overlap\_left}(\alpha, b_1, b_2) \; \equiv \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1}-2 \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1} \right) \; \vee \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1}-1 \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1}+1 \right) \; \vee \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1}-1 \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1}-1 \right) $$

$$ \mathtt{overlap\_right}(\alpha, b_1, b_2) \; \equiv \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1}+2 \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1} \right) \; \vee \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1}+1 \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1}+1 \right) \; \vee \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1}+1 \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1}-1 \right) $$

$$ \mathtt{overlap\_up}(\alpha, b_1, b_2) \; \equiv \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1}-1 \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1}+1 \right) \; \vee \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1} \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1}+2 \right) \; \vee \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1}+1 \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1}+1 \right) $$

$$ \mathtt{overlap\_down}(\alpha, b_1, b_2) \; \equiv \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1}-1 \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1}-1 \right) \; \vee \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1}+1 \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1}-1 \right) \; \vee \; \left( \mathtt{sx}_{b_2} = \mathtt{sx}_{b_1} \; \wedge \; \mathtt{sy}_{b_2} = \mathtt{sy}_{b_1}-2 \right) $$

Podemos agora definir a mudança de rota do controlador para um dado barco:

$$ \mathtt{croute} (\alpha, b) \; \equiv \; \bigwedge_{b' \in \mathcal{B} \setminus b} \mathtt{danger\_boats}(\alpha, b, b') \; \rightarrow \; \big[ \left( \mathtt{\varphi_c} \neq 0 \; \wedge \; \mathtt{overlap\_right} (\alpha, b, b') \right) \; \vee \; \left( \mathtt{\varphi_c} \neq 90 \; \wedge \; \mathtt{overlap\_up} (\alpha, b, b') \right) \; \vee \; \left( \mathtt{\varphi_c} \neq 180 \; \wedge \; \mathtt{overlap\_left} (\alpha, b, b') \right) \vee \; \left( \mathtt{\varphi_c} \neq 270  \; \wedge \; \mathtt{overlap\_down} (\alpha, b, b') \right) \big] $$

In [8]:
def overlap(trace, b1, b2, direc):
    if direc == "left":
        left = And(trace[b2]["sx"] == trace[b1]["sx"]-2, trace[b2]["sy"] == trace[b1]["sy"])
        up = And(trace[b2]["sx"] == trace[b1]["sx"]-1, trace[b2]["sy"] == trace[b1]["sy"]+1)
        down = And(trace[b2]["sx"] == trace[b1]["sx"]-1, trace[b2]["sy"] == trace[b1]["sy"]-1)
        r = Or(left, up, down)
    elif direc == "right":
        right = And(trace[b2]["sx"] == trace[b1]["sx"]+2, trace[b2]["sy"] == trace[b1]["sy"])
        up = And(trace[b2]["sx"] == trace[b1]["sx"]+1, trace[b2]["sy"] == trace[b1]["sy"]+1)
        down = And(trace[b2]["sx"] == trace[b1]["sx"]+1, trace[b2]["sy"] == trace[b1]["sy"]-1)
        r = Or(right, up, down)
    elif direc == "up":
        left = And(trace[b2]["sx"] == trace[b1]["sx"]-1, trace[b2]["sy"] == trace[b1]["sy"]+1)
        up = And(trace[b2]["sx"] == trace[b1]["sx"], trace[b2]["sy"] == trace[b1]["sy"]+2)
        right = And(trace[b2]["sx"] == trace[b1]["sx"]+1, trace[b2]["sy"] == trace[b1]["sy"]+1)
        r = Or(left, up, right)
    elif direc == "down":
        left = And(trace[b2]["sx"] == trace[b1]["sx"]-1, trace[b2]["sy"] == trace[b1]["sy"]-1)
        right = And(trace[b2]["sx"] == trace[b1]["sx"]+1, trace[b2]["sy"] == trace[b1]["sy"]-1)
        down = And(trace[b2]["sx"] == trace[b1]["sx"], trace[b2]["sy"] == trace[b1]["sy"]-2)
        r = Or(left, right, down)
    return r

def c_route(trace, b1):
    r = []
    
    for b2 in trace:
        if "b" in b2 and b1!=b2:
            danger = danger_boats(trace, b1, b2)
            hours = Or(trace["c"]["a"]==0, trace["c"]["a"]==90, trace["c"]["a"]==180, trace["c"]["a"]==270)
            left = And(trace["c"]["a"]!=180, overlap(trace, b1, b2, "left"))
            right = And(trace["c"]["a"]!=0, overlap(trace, b1, b2, "right"))
            up = And(trace["c"]["a"]!=90, overlap(trace, b1, b2, "up"))
            down = And(trace["c"]["a"]!=270, overlap(trace, b1, b2, "down"))
            r.append(Implies(danger, And(hours, left, right, up, down)))
            
    return And(r)

Logo, para a transição untimed to controlador:

$$ \mathtt{cadj} (\alpha, b) \; \equiv \; \mathtt{adjacent} (\alpha', b) \; \wedge \; \left( v_c' = \mathtt{stop} \; \vee \; v_c' = \mathtt{low} \right) $$

$$ \mathtt{cdanger} (\alpha, b) \; \equiv \; \mathtt{danger} (\alpha, b) \; \wedge \; \neg \mathtt{adjacent} (\alpha, b) \; \wedge \; v_c = \mathtt{low} $$

$$ \mathtt{cnotsafe} (\alpha, b) \; \equiv \; \mathtt{croute} (\alpha, b) \; \wedge \; \left( \mathtt{cadj} (\alpha, b) \; \vee \; \mathtt{cdanger} (\alpha, b) \right) $$

$$ \mathtt{csafe} (\alpha, b) \; \equiv \; \neg \mathtt{danger} (\alpha, b) \; \wedge \; v_c = \mathtt{high} \; \wedge \; \bigvee_{\varphi \in \mathcal{R}} \varphi_c = \varphi $$

$$ \mathtt{untimedc} (\alpha, \alpha') \; \equiv \; \bigvee_{b \in \mathcal{B}} \big( \mathtt{sector\_change} (\alpha, \alpha', b) \; \wedge \; \mathtt{sx}_c' = \mathtt{sx}_b' \; \wedge \; \mathtt{sy}_c' = \mathtt{sy}_b' \; \wedge \; \left( \mathtt{cnotsafe} (\alpha', b) \; \vee \; \mathtt{csafe} (\alpha', b) \right) \big) $$

E a função untimed geral é então dada por:

$$ \mathtt{untimed} (\alpha, \alpha') \; \equiv \; \mathtt{untimedc} (\alpha, \alpha') \; \wedge \; \mathtt{untimedb} (\alpha, \alpha', b_1) \; \wedge \; \mathtt{untimedb} (\alpha, \alpha', b_2) \; \wedge \; \mathtt{untimedb} (\alpha, \alpha', b_3) $$

In [9]:
def c_untimed(prev, curr):
    r = []
    for b in prev:
        if "b" in b:
            # Mudança do setor do controlador
            change = sector_change(prev, curr, b)
            sx = curr["c"]["sx"] == curr[b]["sx"]
            sy = curr["c"]["sy"] == curr[b]["sy"]
            
            # Dois navios em setores adjacentes estao em low ou stop
            b_adj = And(adjacent(curr, b), Or(curr["c"]["v"]==M_STOP, curr["c"]["v"]==M_LOW))
            b_danger = And(danger(curr, b), Not(adjacent(curr, b)), curr["c"]["v"]==M_LOW)
            b_nsafe = And(Or(b_adj, b_danger), c_route(curr, b))
            
            # Navio em seguranca
            hour = Or([curr["c"]["a"]==a for a in ALL_ROUTES])
            b_safe = And(Not(danger(curr, b)), curr["c"]["v"]==M_HIGH, hour)
            
            r.append(And(change, sx, sy, Or(b_safe, b_nsafe)))
            
    return Or(r)

untimed = lambda prev, curr: And([b_untimed(prev, curr, b) for b in prev if "b" in b] + [c_untimed(prev, curr)])

## Timed

Uma transição timed de um barco é dada por:

$$ \mathtt{timedb}(\alpha, \alpha', b) \; \equiv \; v_b' = v_b \; \wedge \; \varphi_b' = \varphi_b \; \wedge \; \mathtt{sx}_b' = \mathtt{sx}_b \, \wedge \; \mathtt{sy}_b' = \mathtt{sy}_b \; \wedge \; \mathtt{sx}_b' \cdot L \le x_b' \le \mathtt{sx}_b' \cdot (L + 1) \; \wedge \; \mathtt{sy}_b' \cdot L \le y' \le \mathtt{sy}_b' \cdot (L + 1) \; \wedge \; \big( \mathtt{stop\_stop}(\alpha, \alpha', b) \; \vee \; \mathtt{low\_low}(\alpha, \alpha', b) \; \vee \; \mathtt{high\_high}(\alpha, \alpha', b) \big) $$

In [10]:
def b_timed(prev, curr, b):
    # Não ocorre mudanças de modo
    v = prev[b]["v"] == curr[b]["v"]
    a = prev[b]["a"] == curr[b]["a"]
    sx = prev[b]["sx"] == curr[b]["sx"]
    sy = prev[b]["sy"] == curr[b]["sy"]
    
    # Limitar a posicao do barco com o seu setor
    x_lim = And(curr[b]["x"] >= curr[b]["sx"]*L, curr[b]["x"] <= (curr[b]["sx"]+1)*L)
    y_lim = And(curr[b]["y"] >= curr[b]["sy"]*L, curr[b]["y"] <= (curr[b]["sy"]+1)*L)
    
    # Mudança das variáveis contínuas
    stop = stop_stop(prev, curr, b) 
    low = low_low(prev, curr, b)
    high = high_high(prev, curr, b)
    
    return And(v, a, sx, sy, x_lim, y_lim, Or(stop, high, low))

Para o controlador temos:
    
$$ \mathtt{timedc} (\alpha, \alpha') \; \equiv \; v_c' = v_c \; \wedge \; \varphi_c' = \varphi_c \; \wedge \; \mathtt{sx}_c' = \mathtt{sx}_c \; \wedge \; \mathtt{sy}_c' = \mathtt{sy}_c $$

E por fim a transição timed geral:

$$ \mathtt{timed} (\alpha, \alpha') \; \equiv \; \mathtt{timedc} (\alpha, \alpha') \; \wedge \; \mathtt{timedb} (\alpha, \alpha', b_1) \; \wedge \; \mathtt{timedb} (\alpha, \alpha', b_2) \; \wedge \; \mathtt{timedb} (\alpha, \alpha', b_3) $$

In [11]:
def c_timed(prev, curr):
    # Não ocorre nenhuma mudança no controlador
    v = prev["c"]["v"] == curr["c"]["v"]
    a = prev["c"]["a"] == curr["c"]["a"]
    sx = prev["c"]["sx"] == curr["c"]["sx"]
    sy = prev["c"]["sy"] == curr["c"]["sy"]
    
    return And(v, a, sx, sy)

timed = lambda prev, curr: And([b_timed(prev, curr, b) for b in prev if "b" in b] + [c_timed(prev, curr)])

Deste modo, para ocorrer uma transição mista, devemos ter:
    
$$ \mathtt{mixed}_1 (\alpha, \alpha') \; \equiv \; \mathtt{untimedb} (\alpha, \alpha', b_1) \; \wedge \; \mathtt{untimedb} (\alpha, \alpha', b_2) \; \wedge \; \mathtt{timedb} (\alpha, \alpha', b_3) $$

$$ \mathtt{mixed}_2 (\alpha, \alpha') \; \equiv \; \mathtt{untimedb} (\alpha, \alpha', b_1) \; \wedge \; \mathtt{timedb} (\alpha, \alpha', b_2) \; \wedge \; \mathtt{untimedb} (\alpha, \alpha', b_3) $$

$$ \mathtt{mixed}_3 (\alpha, \alpha') \; \equiv \; \mathtt{timedb} (\alpha, \alpha', b_1) \; \wedge \; \mathtt{untimedb} (\alpha, \alpha', b_2) \; \wedge \; \mathtt{untimedb} (\alpha, \alpha', b_3) $$

$$ \mathtt{mixed}_4 (\alpha, \alpha') \; \equiv \; \mathtt{untimedb} (\alpha, \alpha', b_1) \; \wedge \; \mathtt{timedb} (\alpha, \alpha', b_2) \; \wedge \; \mathtt{timedb} (\alpha, \alpha', b_3) $$

$$ \mathtt{mixed}_5 (\alpha, \alpha') \; \equiv \; \mathtt{timedb} (\alpha, \alpha', b_1) \; \wedge \; \mathtt{untimedb} (\alpha, \alpha', b_2) \; \wedge \; \mathtt{timedb} (\alpha, \alpha', b_3) $$

$$ \mathtt{mixed}_6 (\alpha, \alpha') \; \equiv \; \mathtt{timedb} (\alpha, \alpha', b_1) \; \wedge \; \mathtt{timedb} (\alpha, \alpha', b_2) \; \wedge \; \mathtt{untimedb} (\alpha, \alpha', b_3) $$

E, por fim:

$$ \mathtt{mixed} (\alpha, \alpha') \; \equiv \; \mathtt{untimedc}(\alpha, \alpha') \; \wedge \; \bigvee_{i=1}^{6} \mathtt{mixed}_i (\alpha, \alpha') $$

In [12]:
def mixed(prev, curr):
    uut = And(b_untimed(prev, curr, "b0"), b_untimed(prev, curr, "b1"), b_timed(prev, curr, "b2"))
    utu = And(b_untimed(prev, curr, "b0"), b_timed(prev, curr, "b1"), b_untimed(prev, curr, "b2"))
    tuu = And(b_timed(prev, curr, "b0"), b_untimed(prev, curr, "b1"), b_untimed(prev, curr, "b2"))
    utt = And(b_untimed(prev, curr, "b0"), b_timed(prev, curr, "b1"), b_timed(prev, curr, "b2"))
    tut = And(b_timed(prev, curr, "b0"), b_untimed(prev, curr, "b1"), b_timed(prev, curr, "b2"))
    ttu = And(b_timed(prev, curr, "b0"), b_timed(prev, curr, "b1"), b_untimed(prev, curr, "b2"))
    
    return And(Or(uut, utu, tuu, utt, tut, ttu), c_untimed(prev, curr))

Para verificarmos que os autómatos estão todos sincronizados verificamos se o tempo que passa entre uma transição é o mesmo para todos os barcos:
    
$$ \mathtt{eqd} (\alpha, \alpha') \; \equiv \; t_{b_0}' - t_{b_0} = t_{b_1}' - t_{b_1} \; \wedge \; t_{b_1}' - t_{b_1} = t_{b_2}' - t_{b_2} $$

In [13]:
def eqd(prev, curr):
    t1 = curr["b0"]["t"]-prev["b0"]["t"] == curr["b1"]["t"]-prev["b1"]["t"]
    t2 = curr["b1"]["t"]-prev["b1"]["t"] == curr["b2"]["t"]-prev["b2"]["t"]
        
    return And(t1, t2)

Podemos então definir uma função de transição do autómato híbrido:

$$ \mathtt{trans}(\alpha, \alpha') \; \equiv \; \mathtt{eqd} (\alpha, \alpha') \; \wedge \; \big( \mathtt{untimed} (\alpha, \alpha') \; \vee \; \mathtt{timed}(\alpha, \alpha') \; \vee \; \mathtt{mixed} (\alpha, \alpha') \big) $$

In [14]:
trans = lambda prev, curr: And(eqd(prev, curr), Or(untimed(prev, curr), timed(prev, curr), mixed(prev, curr)))

In [15]:
def model_to_dfs(m):
    """
    Esta função transforma um modelo z3 numa lista de DataFrames pandas.
    Cada elemento desta lista é um DataFrame de um barco.
    """
    
    m_ = {str(elem): m[elem] for elem in m}
    sortedkeys = sorted(m_, key=str.lower)

    prop_names = sorted(list(set([str(elem)[:-1] for elem in sortedkeys])))
    df_dict = {p: [] for p in prop_names}

    for col in df_dict:
        for elem in sortedkeys:
            if col in elem:
                if m_[elem].sort() == RealSort():
                    value = z3tofloat(m_[elem])
                else:
                    value = m_[elem]
                df_dict[col].append(value)

    df_dict1 = {i: {elem: df_dict[elem] for elem in df_dict if f"b{i}" in elem} for i in range(3)}
    df_dict1[3] = {elem: df_dict[elem] for elem in df_dict if "c" in elem}

    dfs = [pd.DataFrame(df_dict1[i]) for i in range(len(df_dict1))]
        
    return dfs
    
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

Podemos verificar que existe colisão num elemento do traço se houver nenhuma sobreposição entre os setores dos barcos:

$$ \mathtt{collision} (\alpha) \; \equiv \; \bigvee_{(b_1, b_2) \in \mathcal{B} \times \mathcal{B}, b_1 \neq b_2 } \left( \mathtt{sx}_{b_1} = \mathtt{sx}_{b_2} \; \wedge \; \mathtt{sy}_{b_1} = \mathtt{sy}_{b_2} \right) $$

A verificação de existirem colisões num traço de tamanho `k` é finalmente dada pela condição:
    
$$ \mathtt{collisions} (\alpha, k) \; \equiv \; \mathtt{init} (\alpha_0) \; \wedge \; \bigwedge_{i=0}^{k-2} \mathtt{trans} (\alpha_i, \alpha_{i+1}) \; \wedge \; \bigvee_{i=0}^{k-1} \mathtt{collision} (\alpha_i) $$

Se esta condição for `unsat`, então os barcos navegam sem colisões.

In [17]:
def collision(trace):
    r = []
    for b1, b2 in product(*[trace, trace]):
        if "b" in b1 and "b" in b2 and b1!=b2:
            sx = trace[b1]["sx"]==trace[b2]["sx"]
            sy = trace[b1]["sy"]==trace[b2]["sy"]
            r.append(And(sx, sy))
            
    return Or(r)

def gen_trace(declare, init, trans, f, k):
    solver = Solver()
    trace = {i: declare(i) for i in range(k)}
    solver.add(init(trace[0]))
    
    for i in range(k-1):
        solver.add(trans(trace[i], trace[i+1]))
        
    solver.add(Or([f(trace[i]) for i in range(k)]))
        
    if solver.check() == unsat:
        print("No collisions can happen")
        r = None
    else:
        print("Counter example")
        m = solver.model()
        r = model_to_dfs(m)
        display_side_by_side(*r)
        
    return r

tr = gen_trace(declare, init, trans, collision, 20)

No collisions can happen
